In [1]:
import pandas as pd

In [2]:
target_date_str = '20150302'
target_datetime = pd.to_datetime(target_date_str, format='%Y%m%d')

In [20]:
orders_df = pd.read_csv(f'OrderLog{target_date_str}/OrderLog{target_date_str}.txt', index_col='NO')

In [17]:
trades_df = pd.read_csv(f'OrderLog{target_date_str}/TradeLog{target_date_str}.txt')

## 1. Preprocessing

### 1.1 Add DATETIME column with datetime

In [30]:
orders_df['DATETIME'] = pd.to_datetime(target_date_str + orders_df['TIME'].astype(str).str.zfill(9), format='%Y%m%d%H%M%S%f')

In [31]:
orders_df[(orders_df.SECCODE == 'SBER') & (orders_df.TIME > 100000000) & (orders_df.TIME < 100002000)].DATETIME

NO
10453   2015-03-02 10:00:00.071
10455   2015-03-02 10:00:00.073
10457   2015-03-02 10:00:00.095
10460   2015-03-02 10:00:00.110
10466   2015-03-02 10:00:00.139
                  ...          
13079   2015-03-02 10:00:01.904
13103   2015-03-02 10:00:01.931
13104   2015-03-02 10:00:01.931
13105   2015-03-02 10:00:01.931
13130   2015-03-02 10:00:01.960
Name: DATETIME, Length: 243, dtype: datetime64[ns]

In [32]:
trades_df['DATETIME'] = pd.to_datetime(target_date_str + trades_df['TIME'].astype(str).str.zfill(6), format='%Y%m%d%H%M%S')

In [33]:
trades_df.DATETIME

0        2015-03-02 10:00:00
1        2015-03-02 10:00:00
2        2015-03-02 10:00:00
3        2015-03-02 10:00:00
4        2015-03-02 10:00:00
                 ...        
409249   2015-03-02 18:39:58
409250   2015-03-02 18:39:58
409251   2015-03-02 18:39:58
409252   2015-03-02 18:39:58
409253   2015-03-02 18:39:58
Name: DATETIME, Length: 409254, dtype: datetime64[ns]

In [34]:
orders_df[(orders_df.SECCODE == 'SBER') & (orders_df.ACTION == 2)]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATETIME
NO,,,,,,,,,,
10603,SBER,B,100000550,10567,2,0.00,100,2.460135e+09,76.34,2015-03-02 10:00:00.550
10604,SBER,S,100000550,10465,2,76.34,100,2.460135e+09,76.34,2015-03-02 10:00:00.550
10611,SBER,B,100000566,10492,2,76.33,10000,2.460135e+09,76.33,2015-03-02 10:00:00.566
10612,SBER,S,100000566,10572,2,76.33,10000,2.460135e+09,76.33,2015-03-02 10:00:00.566
10645,SBER,B,100000615,10492,2,76.33,50,2.460135e+09,76.33,2015-03-02 10:00:00.615
...,...,...,...,...,...,...,...,...,...,...
14200411,SBER,S,183958468,6920196,2,0.00,190,2.460557e+09,76.15,2015-03-02 18:39:58.468
14200412,SBER,B,183958468,6919675,2,76.12,10,2.460557e+09,76.12,2015-03-02 18:39:58.468
14200413,SBER,S,183958468,6920196,2,0.00,10,2.460557e+09,76.12,2015-03-02 18:39:58.468


In [74]:
sber_orders_df = orders_df[(orders_df.SECCODE == 'SBER')]
sber_group = sber_orders_df.groupby(by='ORDERNO').count()
sber_group = sber_group[sber_group.SECCODE > 4]

In [78]:
sber_orders_df[sber_orders_df.ORDERNO == 6692]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATETIME
NO,,,,,,,,,,
6692,SBER,S,100000000,6692,1,76.96,10000,NaN,NaN,2015-03-02 10:00:00.000
3227109,SBER,S,105915314,6692,2,76.96,870,2.460211e+09,76.96,2015-03-02 10:59:15.314
3227123,SBER,S,105915317,6692,2,76.96,500,2.460211e+09,76.96,2015-03-02 10:59:15.317
3227128,SBER,S,105915318,6692,2,76.96,900,2.460211e+09,76.96,2015-03-02 10:59:15.318
3227134,SBER,S,105915321,6692,2,76.96,1200,2.460211e+09,76.96,2015-03-02 10:59:15.321
3227202,SBER,S,105915338,6692,2,76.96,500,2.460211e+09,76.96,2015-03-02 10:59:15.338
3227212,SBER,S,105915344,6692,2,76.96,1200,2.460211e+09,76.96,2015-03-02 10:59:15.344
3227215,SBER,S,105915347,6692,2,76.96,1200,2.460211e+09,76.96,2015-03-02 10:59:15.347
3227222,SBER,S,105915351,6692,2,76.96,1200,2.460211e+09,76.96,2015-03-02 10:59:15.351


In [80]:
orders_df

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATETIME
NO,,,,,,,,,,
1,RNAV,S,100000000,1,1,7.000,7900,NaN,NaN,2015-03-02 10:00:00.000
2,AFLT,S,100000000,2,1,42.000,100,NaN,NaN,2015-03-02 10:00:00.000
3,AFLT,S,100000000,3,1,40.200,200,NaN,NaN,2015-03-02 10:00:00.000
4,ALRS,S,100000000,4,1,69.960,200,NaN,NaN,2015-03-02 10:00:00.000
5,BLNG,B,100000000,5,1,4.485,200,NaN,NaN,2015-03-02 10:00:00.000
...,...,...,...,...,...,...,...,...,...,...
14203557,MGNT,S,184459362,6877462,0,11700.000,1,NaN,NaN,2015-03-02 18:44:59.362
14203558,FXDE,B,184459605,6920668,0,1971.000,3000,NaN,NaN,2015-03-02 18:44:59.605
14203559,FXDE,S,184459636,6920669,0,1975.500,11990,NaN,NaN,2015-03-02 18:44:59.636


In [86]:
orders_df[orders_df.ORDERNO == 6540698]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATETIME
NO,,,,,,,,,,
13402140,OPIN,B,182047306,6540698,1,209.0,59,NaN,NaN,2015-03-02 18:20:47.306
14203560,OPIN,B,184459798,6540698,0,209.0,59,NaN,NaN,2015-03-02 18:44:59.798


In [11]:
orders_df.VOLUME.dtype

dtype('int64')

In [24]:
from collections import defaultdict
from dataclasses import dataclass


@dataclass
class Order:
    price: float
    volume: int


class OrderBook:
    def __init__(self):
        self._buy_book = defaultdict(dict)
        self._sell_book = defaultdict(dict)
        self._top_buy = None
        self._bottom_sell = None
        self._price_delta = 0.01
    
    def get_best_bid(self) -> float | None:
        return self._top_buy
    
    def get_best_ask(self):
        return self._bottom_sell
    
    def add_buy_order(self, price: float, orderno: int, volume: int):
        if self._top_buy is None or price > self._top_buy:
            self._top_buy = price
        self._buy_book[price][orderno] = Order(price, volume)
    
    def add_sell_order(self, price: float, orderno: int, volume: int):
        if self._bottom_sell is None or price < self._bottom_sell:
            self._bottom_sell = price
        self._sell_book[price][orderno] = Order(price, volume)
    
    def cancel_buy_order(self, price: float, orderno: int):
        self._buy_book[price].pop(orderno)
        if len(self._buy_book[price]) == 0:
            self._buy_book.pop(price)
        while self._buy_book.get(self._top_buy) is None:
            self._top_buy -= self._price_delta
    
    def cancel_sell_order(self, price: float, orderno: int):
        self._sell_book[price].pop(orderno)
        if len(self._sell_book[price]) == 0:
            self._sell_book.pop(price)
        while self._sell_book.get(self._bottom_sell) is None:
            self._bottom_sell += self._price_delta
    
    def buy(self, price: float, orderno: int, volume: int):
        if self._sell_book[price][orderno].volume == volume:
            self._sell_book[price].pop(orderno)
            if len(self._sell_book[price]) == 0:
                self._sell_book.pop(price)
        else:
            self._sell_book[price][orderno].volume -= volume
    
    def sell(self, price: float, orderno: int, volume: int):
        if self._buy_book[price][orderno].volume == volume:
            self._buy_book[price].pop(orderno)
            if len(self._buy_book[price]) == 0:
                self._buy_book.pop(price)
        else:
            self._buy_book[price][orderno].volume -= volume


In [80]:
orders_df

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATETIME
NO,,,,,,,,,,
1,RNAV,S,100000000,1,1,7.000,7900,NaN,NaN,2015-03-02 10:00:00.000
2,AFLT,S,100000000,2,1,42.000,100,NaN,NaN,2015-03-02 10:00:00.000
3,AFLT,S,100000000,3,1,40.200,200,NaN,NaN,2015-03-02 10:00:00.000
4,ALRS,S,100000000,4,1,69.960,200,NaN,NaN,2015-03-02 10:00:00.000
5,BLNG,B,100000000,5,1,4.485,200,NaN,NaN,2015-03-02 10:00:00.000
...,...,...,...,...,...,...,...,...,...,...
14203557,MGNT,S,184459362,6877462,0,11700.000,1,NaN,NaN,2015-03-02 18:44:59.362
14203558,FXDE,B,184459605,6920668,0,1971.000,3000,NaN,NaN,2015-03-02 18:44:59.605
14203559,FXDE,S,184459636,6920669,0,1975.500,11990,NaN,NaN,2015-03-02 18:44:59.636


### ACTION
0=Cancel order  
1=Create order  
2=Trade  

In [ ]:
best_bid = []
best_ask = []
cur_best_bid = None
cur_best_ask = None

order_book = OrderBook()
for idx, row in orders_df[(orders_df.SECCODE == 'SBER')].iterrows():
    buysell = row['BUYSELL']
    orderno = row['ORDERNO']
    price = row['PRICE']
    volume = row['VOLUME']
    action = row['ACTION']
    if row['ACTION'] == 0:
        order_book.add_buy_order()
    elif row['ACTION'] == 1:
        ...
    elif row['ACTION'] == 2:
        ...
    else:
        raise ValueError(f'Unknown action: {row["ACTION"]}')


In [72]:
trades_df[trades_df.SECCODE == 'SBER']

,TRADENO,SECCODE,TIME,BUYORDERNO,SELLORDERNO,PRICE,VOLUME,DATETIME
11,2460134743,SBER,100000,10567,10465,76.34,100,2015-03-02 10:00:00
12,2460134744,SBER,100000,10492,10572,76.33,10000,2015-03-02 10:00:00
13,2460134745,SBER,100000,10492,10601,76.33,50,2015-03-02 10:00:00
16,2460134749,SBER,100000,10647,10465,76.34,900,2015-03-02 10:00:00
17,2460134750,SBER,100000,10647,9868,76.35,7100,2015-03-02 10:00:00
...,...,...,...,...,...,...,...,...
409242,2460556946,SBER,183958,6920165,6920122,76.18,10,2015-03-02 18:39:58
409244,2460556948,SBER,183958,6920120,6920192,76.15,10,2015-03-02 18:39:58
409245,2460556949,SBER,183958,6920120,6920196,76.15,190,2015-03-02 18:39:58
409246,2460556950,SBER,183958,6919675,6920196,76.12,10,2015-03-02 18:39:58


In [59]:
trades_df[trades_df.SECCODE == 'SBER'].TIME.max()

np.int64(183958)

In [25]:
orders_df[orders_df['SECCODE'] == 'SBER'].shape

(1994350, 9)

In [14]:
trades_df[trades_df['SECCODE'] == 'SBER'].shape

(82364, 7)

In [49]:
orders_df.TIME.max()

np.int64(184459956)

In [33]:
orders_df[orders_df.SECCODE == 'SBER'].TRADENO.notna().sum()

np.int64(204538)

In [ ]:
full_channel_df.time = pd.to_datetime(full_channel_df.time)
full_channel_df['timestamp'] = full_channel_df['time'].astype('int64')

ticker_df.time = pd.to_datetime(ticker_df.time)
ticker_df['timestamp'] = ticker_df['time'].astype('int64')